In [8]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import seaborn as sns
sns.set(style="ticks", color_codes=True)

data = pd.read_csv('Hospital_Readmissions_Reduction_Program.csv')

too_few = data[data['Number of Readmissions'] == 'Too Few to Report']
is_null = data.loc[pd.isnull(data['Number of Discharges']) 
                   & pd.isnull(data['Number of Readmissions'])]
not_null = data.loc[pd.notnull(data['Number of Discharges']) 
                    | pd.notnull(data['Number of Readmissions'])]
not_zero = not_null[not_null['Number of Discharges'] != 0]
clean_data = not_zero.replace('Too Few to Report', np.NaN)
clean_data['Number of Readmissions'] = clean_data['Number of Readmissions'].astype(float)

clean_data['Readmissions Rate'] = clean_data['Number of Readmissions'] / clean_data['Number of Discharges'] *100
clean_data['Actual Excess Readmission Ratio'] = clean_data['Readmissions Rate'] / clean_data['Expected Readmission Rate']

temp_ = clean_data.groupby('Facility ID').count().reset_index().copy()
type_fac = pd.DataFrame({'Facility ID': temp_['Facility ID'],
                         'Number of Departments': temp_['Facility Name']})
conditions = [type_fac['Number of Departments'] > 4,
             (type_fac['Number of Departments'] >= 3) & (type_fac['Number of Departments'] <= 4),
             type_fac['Number of Departments'] < 3]
values = ['General', 'Neither', 'Specialized']
type_fac['Type'] = np.select(conditions,values)
data_dept = pd.merge(clean_data, type_fac, on='Facility ID', how='left')

def filter(data, col_name, filter_value):
    return data[data[col_name] == filter_value]

diagnosis_lst = ['READM-30-AMI-HRRP',
                 'READM-30-CABG-HRRP',
                 'READM-30-COPD-HRRP',
                 'READM-30-HF-HRRP',
                 'READM-30-HIP-KNEE-HRRP',
                 'READM-30-PN-HRRP']

type_lst = ['General','Neither','Specialized']

In [3]:
d= {}
for name in diagnosis_lst:
    d[name] = filter(data_dept, 'Measure Name', name)

In [6]:
d['READM-30-AMI-HRRP'];

In [14]:
def discharges_readmissions_by_type(measure_name):
    by_type_lst = []
    for type in type_lst:
        discharges = filter(d[measure_name], 'Type', type)['Number of Discharges'].sum()
        readmissions = filter(d[measure_name], 'Type', type)['Number of Readmissions'].sum()
        rate = readmissions / discharges
        by_type_lst.append([discharges, readmissions, rate])
    return by_type_lst

In [15]:
discharges_readmissions_by_type('READM-30-AMI-HRRP')

[[440926.0, 67121.0, 0.1522273578786463],
 [9729.0, 1807.0, 0.18573337444752802],
 [357.0, 41.0, 0.11484593837535013]]

In [19]:
discharges_readmissions_by_type('READM-30-HIP-KNEE-HRRP')

[[722829.0, 29623.0, 0.04098203032805823],
 [28162.0, 1196.0, 0.0424685746750941],
 [81244.0, 2446.0, 0.030106838658854808]]

In [21]:
total_discharged = discharges_readmissions_by_type('READM-30-HIP-KNEE-HRRP')[0][0] + discharges_readmissions_by_type('READM-30-HIP-KNEE-HRRP')[2][0]
    
total_readmitted = discharges_readmissions_by_type('READM-30-HIP-KNEE-HRRP')[0][1] + discharges_readmissions_by_type('READM-30-HIP-KNEE-HRRP')[2][1]

shared_sample_freq = total_readmitted / total_discharged
denominator = discharges_readmissions_by_type('READM-30-HIP-KNEE-HRRP')[0][0] * discharges_readmissions_by_type('READM-30-HIP-KNEE-HRRP')[2][0]

shared_sample_variance = total_discharged * (shared_sample_freq * (1 - shared_sample_freq)) / denominator

difference_in_proportions = stats.norm(0, np.sqrt(shared_sample_variance))

In [22]:
difference_in_sample_proportions = discharges_readmissions_by_type('READM-30-HIP-KNEE-HRRP')[0][2] - discharges_readmissions_by_type('READM-30-HIP-KNEE-HRRP')[2][2]

In [23]:
difference_in_sample_proportions

0.010875191669203421

In [25]:
p_value = 1 - difference_in_proportions.cdf(difference_in_sample_proportions)
p_value

0.0

In [26]:
a = filter(filter(data_dept,'Measure Name','READM-30-HIP-KNEE-HRRP'),'Type', 'Specialized')['Readmissions Rate']
b = filter(filter(data_dept,'Measure Name','READM-30-HIP-KNEE-HRRP'),'Type', 'General')['Readmissions Rate']
stats.ttest_ind(a,b, equal_var=False,nan_policy='omit')

Ttest_indResult(statistic=-8.134923457842765, pvalue=1.314373319312116e-12)

In [41]:
np.var(a), np.var(b), np.var(a) + np.var(b)

(1.2946566533421437, 3.16924663711317, 4.463903290455313)

In [42]:
difference_in_proportions2 = stats.norm(0, np.sqrt(np.var(a) + np.var(b)))

In [43]:
p_value2 = 1 - difference_in_proportions2.cdf(difference_in_sample_proportions)
p_value

0.0

In [27]:
dis_sp=filter(filter(data_dept,'Measure Name','READM-30-HIP-KNEE-HRRP'),'Type', 'Specialized')['Number of Discharges'].sum()
read_sp=filter(filter(data_dept,'Measure Name','READM-30-HIP-KNEE-HRRP'),'Type', 'Specialized')['Number of Readmissions'].sum()

dis_g=filter(filter(data_dept,'Measure Name','READM-30-HIP-KNEE-HRRP'),'Type', 'General')['Number of Discharges'].sum()
read_g=filter(filter(data_dept,'Measure Name','READM-30-HIP-KNEE-HRRP'),'Type', 'General')['Number of Readmissions'].sum()

dis_sp, read_sp, read_sp/dis_sp, dis_g, read_g, read_g/dis_g

(81244.0, 2446.0, 0.030106838658854808, 722829.0, 29623.0, 0.04098203032805823)

In [33]:
binomial = stats.binom(n=81244, p=read_sp/dis_sp)

In [34]:
read_g/dis_g * 81244

3329.5440719727626

In [35]:
prob_equal_or_more_extreme = 1 - binomial.cdf(3329)
prob_equal_or_more_extreme

1.1102230246251565e-16

In [45]:
data_dept

,Facility Name,Facility ID,State,Measure Name,Number of Discharges,Footnote,Excess Readmission Ratio,Predicted Readmission Rate,Expected Readmission Rate,Number of Readmissions,Start Date,End Date,Readmissions Rate,Actual Excess Readmission Ratio,Number of Departments,Type
0,SOUTHEAST ALABAMA MEDICAL CENTER,10001,AL,READM-30-HIP-KNEE-HRRP,301.0,NaN,1.1787,5.5863,4.7392,20.0,07/01/2015,06/30/2018,6.644518,1.402034,6,General
1,SOUTHEAST ALABAMA MEDICAL CENTER,10001,AL,READM-30-CABG-HRRP,279.0,NaN,1.2361,14.5943,11.8065,46.0,07/01/2015,06/30/2018,16.487455,1.396473,6,General
2,SOUTHEAST ALABAMA MEDICAL CENTER,10001,AL,READM-30-AMI-HRRP,742.0,NaN,1.0446,15.2935,14.6404,116.0,07/01/2015,06/30/2018,15.633423,1.067828,6,General
3,SOUTHEAST ALABAMA MEDICAL CENTER,10001,AL,READM-30-HF-HRRP,1114.0,NaN,1.0453,22.3772,21.4082,252.0,07/01/2015,06/30/2018,22.621185,1.056660,6,General
4,SOUTHEAST ALABAMA MEDICAL CENTER,10001,AL,READM-30-COPD-HRRP,495.0,NaN,1.0249,18.6162,18.1637,94.0,07/01/2015,06/30/2018,18.989899,1.045486,6,General
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14066,HOUSTON METHODIST THE WOODLANDS HOSPITAL,670122,TX,READM-30-AMI-HRRP,NaN,NaN,1.0168,16.6858,16.4107,NaN,07/01/2015,06/30/2018,NaN,NaN,5,General
14067,HOUSTON METHODIST THE WOODLANDS HOSPITAL,670122,TX,READM-30-COPD-HRRP,NaN,NaN,1.0240,18.6577,18.2205,NaN,07/01/2015,06/30/2018,NaN,NaN,5,General
14068,HOUSTON METHODIST THE WOODLANDS HOSPITAL,670122,TX,READM-30-HF-HRRP,79.0,NaN,1.0489,21.6269,20.6191,20.0,07/01/2015,06/30/2018,25.316456,1.227816,5,General
14069,HOUSTON METHODIST THE WOODLANDS HOSPITAL,670122,TX,READM-30-HIP-KNEE-HRRP,NaN,NaN,1.1380,4.5980,4.0405,NaN,07/01/2015,06/30/2018,NaN,NaN,5,General
